In [ ]:
import numpy as np
import matplotlib

matplotlib.use("agg")

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


class HistogramNormalize(matplotlib.colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        matplotlib.colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(
            np.searchsorted(self.sorted_data, value) / len(self.sorted_data)
        )
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm


golden_mean = (np.sqrt(5) - 1) / 2  # Aesthetic ratio
fig_width_pt = 246.0  # Columnwidth
inches_per_pt = 1 / 72.27  # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean  # height in inches
fig_size = [fig_width, fig_height]

mpl_params = {
    "backend": "ps",
    "axes.labelsize": 13,
    "font.size": 13,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "text.usetex": True,
    "figure.figsize": fig_size,
    "font.family": "serif",
    "font.serif": "Computer Modern Roman",
    "legend.frameon": True,
    "savefig.dpi": 300,
}

plt.rcParams.update(mpl_params)
plt.rc("text.latex", preamble=[r"\usepackage{xfrac}", r"\usepackage{siunitx}"])


def gist_heat_r_transparent():
    import matplotlib.cm
    import matplotlib.colors as mcolors

    colors = matplotlib.cm.gist_heat_r(np.linspace(0, 1, 128))
    colors[:, 3] = np.linspace(0, 1, 128).tolist()
    gist_heat_r_transparent = mcolors.LinearSegmentedColormap.from_list(
        "gist_heat_r_transparent", colors
    )
    return gist_heat_r_transparent


gray_r = matplotlib.cm.gray_r
gray_r.set_bad("#e0e0e0")


def num_to_latex_exp(x, only_exp=True, N_digits=1):
    if not only_exp:
        if N_digits == 1:
            num, exponent = f"{x:.0e}".split("e")
        elif N_digits == 2:
            num, exponent = f"{x:.1e}".split("e")
        return rf"{num} \times 10^{{{int(exponent)}}}"
    else:
        exponent = np.floor(np.log10(np.abs(x))).astype(int)
        return f"10^{{{int(exponent)}}}"
    

def color_spline(ax, c):
    ax.spines["bottom"].set_color(c)
    ax.spines["top"].set_color(c)
    ax.spines["right"].set_color(c)
    ax.spines["left"].set_color(c)
    ax.tick_params(axis="x", colors=c)
    ax.tick_params(axis="y", colors=c)
    for tick in ax.get_yticklabels():
        tick.set_color("k")
    for tick in ax.get_xticklabels():
        tick.set_color("k")

In [ ]:
from learners_file import *

In [ ]:
adaptive.notebook_extension()

In [ ]:
learner = learners[-1]

lead_pars = learner.function.keywords["lead_pars"]
params = learner.function.keywords["params"]
lead = phase_diagram.make_lead(**dict(lead_pars, wraparound=True)).finalized()
B_xs = np.linspace(*learner.bounds[0], 100)
phase_bounds = []
for B_x in B_xs:
    params["B_x"] = B_x
    phase_bounds.append(
        phase_diagram.find_phase_bounds(
            lead, phase_diagram.parse_params(params), num_bands=50, mu_param="mu_lead"
        )[::2]
    )

data = learner.plot(n=200).Image.I.data

In [ ]:
def bands(k_x, mu, B_x, params=params, lead=lead):
    H = lead.hamiltonian_submatrix(params=dict(params, B_x=B_x, k_x=k_x, mu_lead=mu))
    return np.linalg.eigvalsh(H)

mu_mid = phase_bounds[len(B_xs)//2][0]  # first_phase_bound
B_x = B_xs[len(B_xs)//2]
eps = 1
mu_before = mu_mid - eps
mu_after = mu_mid + eps
ks = np.linspace(-0.5, 0.5, 301)
E_mid = np.array([bands(k, mu=mu_mid, B_x=B_x) for k in ks])
E_before = np.array([bands(k, mu=mu_before, B_x=B_x) for k in ks])
E_after = np.array([bands(k, mu=mu_after, B_x=B_x) for k in ks])


In [ ]:
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt

fig, axs = plt.subplots(
    2, 2, sharex=False, sharey=False, figsize=(2 * fig_width, 3 * fig_height)
)
plt.subplots_adjust(
    bottom=0.2, left=0.125, right=0.85, top=0.8, hspace=0.5, wspace=0.32
)
(ax1, ax2), (ax3, ax4) = axs


def plot_learner_on_ax(ax, learner):
    im = learner.plot(n=400).Image.I
    l, b, r, t = im.bounds.lbrt()
    data = im.data
    data[data < 0] = data[data < 0] * -1
    return ax.imshow(
        im.data,
        extent=(l, r, b, t),
        aspect="auto",
        cmap="viridis",
        norm=HistogramNormalize(data, vmin=0, vmax=0.5),
    )


for c, mu in zip(["C2", "C3", "C4"], [mu_before, mu_mid, mu_after]):
    ax1.scatter([B_x], [mu], edgecolors=c, zorder=4, facecolors="none")

color_spline(ax2, "C2")
color_spline(ax3, "C3")
color_spline(ax4, "C4")
ax2.plot(ks, E_before, c="k")
ax3.plot(ks, E_mid, c="k")
ax4.plot(ks, E_after, c="k")

for ax in [ax2, ax3, ax4]:
    ax.set_ylim(-2.3, 2.3)
    ax.set_xlabel("$a k_x$")
    ax.set_ylabel("$E$ (meV)")
    xvals = [-np.pi / 4, 0, np.pi / 4]
    xlabels = ["$-\pi/4$", r"$0$", "$\pi/4$"]
    ax.set_xticks(xvals)
    ax.set_xticklabels(xlabels)


ax1.plot(B_xs, phase_bounds, c="C6", lw=1, ls="--")
im = plot_learner_on_ax(ax1, learner)
ax1.set_ylim(0, 15)
ax1.set_xlabel("$B_x$ (T)")
ax1.set_ylabel("$\mu$ (meV)")

for i, ax in enumerate(axs.reshape(-1)):
    label = "abcdef"[i]
    ax.text(
        0.995,
        0.97,
        f"$\mathrm{{({label})}}$",
        transform=ax.transAxes,
        verticalalignment="top",
        horizontalalignment="right",
        fontsize=14,
        color="black" if i >= 1 else "white",
    )
    if i == 0:
        ax.set_title("Phase diagram")
    else:
        mu_text = r"$\mu={:.0f}$ meV".format([mu_before, mu_mid, mu_after][i - 1])
        ax.set_title(mu_text)

cb = fig.colorbar(im, ax=ax1, extend="max")
cbar_ticks = [0, 0.5]
cb.set_ticks(cbar_ticks)
cb.set_label(r"$E_{\mathrm{gap}}$ (meV)", labelpad=-13)

plt.savefig("figures/phase_diagram_with_bands.pdf", bbox_inches="tight", transparent=True)
plt.show()

# generalized ev problem

In [ ]:
learner = learners[-1]
lead_pars = learner.function.keywords["lead_pars"]
params = learner.function.keywords["params"]
params = dict(phase_diagram.parse_params(params), B_x=B_x)
lead = phase_diagram.make_lead(**dict(lead_pars, wraparound=True)).finalized()
phase_bounds = phase_diagram.find_phase_bounds(
    lead, params, num_bands=50, mu_param="mu_lead"
)[::2]
mus = np.linspace(0, 25, 201)
Es = []
for params["mu_lead"] in mus:
    H = lead.hamiltonian_submatrix(params=dict(params, k_x=0))
    Es.append(np.linalg.eigvalsh(H))
Es = np.array(Es)

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, 2 * fig_height))
ax.plot(mus, Es, c="k", lw=0.5)
y_min_max = (-15, 15)
ax.set_xlim(mus[0], mus[-1])
ax.set_ylim(y_min_max)
ax.hlines(0, mus[0], mus[-1], color="C1", alpha=0.3)
ax.scatter(
    phase_bounds,
    np.zeros_like(phase_bounds),
    edgecolors="C1",
    alpha=1,
    zorder=10,
    facecolors="none",
)

mu_k = 2
i = np.abs(mus - mu_k).argmin()

ax.vlines(mu_k, *y_min_max, color="C2", alpha=0.3)
ax.scatter(
    mu_k * np.ones_like(Es[i]),
    Es[i],
    edgecolors="C2",
    alpha=1,
    zorder=10,
    facecolors="none",
)
ax.set_xlabel("$\mu$ (meV)")
ax.set_ylabel("$E(k=0)$ (meV)")

# To specify the number of ticks on both or any single axes
plt.locator_params(axis='y', nbins=4)
plt.locator_params(axis='x', nbins=4)
plt.savefig("figures/generalized_ev_problem.pdf", bbox_inches="tight", transparent=True)


# Limitations plot

In [ ]:
import phase_diagram
import kwant
import re
import kwant.continuum
from types import SimpleNamespace
from scipy.constants import physical_constants

import scipy.constants


def get_hamiltonian(subs, dim=1, with_holes=True):
    ham = (
        "(0.5 * hbar**2 * (k_x**2 + k_y**2 + k_z**2) / m_eff - mu) * kron(sigma_0, sigma_z)"
        "+ alpha * (k_y * kron(sigma_x, sigma_z) - k_x * kron(sigma_y, sigma_z))"
        "+ 0.5 * g * mu_B * (B_x * kron(sigma_x, sigma_0) + B_y * kron(sigma_y, sigma_0) + B_z * kron(sigma_z, sigma_0))"
        "+ Delta * kron(sigma_0, sigma_x)"
    )

    if dim == 1:
        ham = ham.replace("k_y", "0")
        ham = ham.replace("k_z", "0")
    elif dim == 2:
        ham = ham.replace("k_z", "0")

    if not with_holes:
        ham = re.sub(r"kron\((sigma_[xyz0]), sigma_[xzy0]\)", r"\1", ham)
    return ham


def make_lead(a, r=None, dim=1, with_holes=True, subs={}):
    symmetry = kwant.TranslationalSymmetry((a, 0, 0)[:dim])
    lead = kwant.Builder(symmetry)
    ham = get_hamiltonian({}, dim, with_holes)
    template = kwant.continuum.discretize(ham, grid=a)
    lead.fill(template, lambda s: True, (0,))
    lead = kwant.wraparound.wraparound(lead).finalized()
    return lead


# (Fundamental) constants definitions in nm and meV.
constants = {
    "m_eff": 0.015 * scipy.constants.m_e / scipy.constants.eV / (1e9) ** 2 * 1e3,
    "phi_0": 2 * physical_constants["mag. flux quantum"][0] * (1e9) ** 2,
    "mu_B": physical_constants["Bohr magneton in eV/T"][0] * 1e3,
    "hbar": scipy.constants.hbar / scipy.constants.eV * 1e3,
}

params = dict(alpha=20, B_x=10, B_y=1, B_z=0, Delta=0.0, g=50, mu=10, V=0, **constants)


fig, axs = plt.subplots(2, figsize=(fig_width, 2 * fig_height), sharex=True)
plt.subplots_adjust(hspace=0.3)
for i, ax in enumerate(axs):
    a = [10, 5][i]
    with_holes = False
    lead = make_lead(a, with_holes=with_holes)
    hamiltonian = get_hamiltonian({}, with_holes=with_holes)
    h_k = kwant.continuum.lambdify(hamiltonian, locals=params)
    k_cont = np.linspace(-4, 4, 501)
    e_cont = [scipy.linalg.eigvalsh(h_k(k_x=ki)) for ki in k_cont]

    def h_k(k_x):
        return lead.hamiltonian_submatrix(params=dict(params, k_x=k_x))

    k_tb = np.linspace(-np.pi / a, np.pi / a, 501)
    e_tb = [scipy.linalg.eigvalsh(h_k(k_x=a * k)) for k in k_tb]
    ax.set_title(label=f"$a={a}$ nm")
    ax.plot(k_cont, e_cont, c="C2")
    ax.plot(k_tb, e_tb, c="C1")
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-30, 100)
    ax.set_ylabel("$E$ (meV)")
    if i == 1:
        ax.set_xlabel("$k$ $(1/a)$")

    ax.hlines(0, -1, 1, linestyles="--", alpha=0.5, linewidth=0.5)
box = axs[0].get_position()

axs[0].legend(
    handles=(line1, line2),
    labels=["continiuum", "tight-binding"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
)
plt.savefig(
    "figures/discretization-approximation.pdf", bbox_inches="tight", transparent=True
)

# Compare different spectra

In [ ]:
from copy import copy
import kwant
from types import SimpleNamespace

pauli = SimpleNamespace(
    s0=np.array([[1.0, 0.0], [0.0, 1.0]]),
    sx=np.array([[0.0, 1.0], [1.0, 0.0]]),
    sy=np.array([[0.0, -1j], [1j, 0.0]]),
    sz=np.array([[1.0, 0.0], [0.0, -1.0]]),
)

pauli.s0s0 = np.kron(pauli.s0, pauli.s0)
pauli.s0sy = np.kron(pauli.s0, pauli.sy)
pauli.s0sz = np.kron(pauli.s0, pauli.sz)
pauli.szsx = np.kron(pauli.sz, pauli.sx)


def bhz(w):
    """Translationally invariant BHZ system with a infinite or fixed width w."""
    lat = kwant.lattice.square()

    def ribbon_shape(pos):
        (x, y) = pos
        return 0 <= y < w

    def onsite(site, B, D, M, ez_y):
        return (
            (M - 4 * B) * pauli.s0sz
            - 4 * D * pauli.s0s0
            + ez_y * np.kron(pauli.sy, (pauli.s0 + pauli.sz) / 2)
        )

    def hopx(site1, site2, A, B, D):
        return B * pauli.s0sz + D * pauli.s0s0 + 1j * A * pauli.szsx

    def hopy(site1, site2, A, B, D):
        return B * pauli.s0sz + D * pauli.s0s0 - 1j * A * pauli.s0sy

    sym = kwant.TranslationalSymmetry((1, 0))
    syst = kwant.Builder(sym)
    syst[lat.shape(ribbon_shape, (0, 0))] = onsite

    syst[kwant.HoppingKind((1, 0), lat)] = hopx
    syst[kwant.HoppingKind((0, 1), lat)] = hopy
    return syst


syst = kwant.wraparound.wraparound(bhz(w=20)).finalized()

def_params = dict(ez_y=0.0, A=0.5, B=1.0, D=0.0,)
p_topo = dict(def_params, M=0.2)
p_triv = dict(def_params, M=-0.2)
k_xs = np.linspace(-np.pi / 3, np.pi / 3, 51)


def bands(p):
    return np.array(
        [
            np.linalg.eigvalsh(syst.hamiltonian_submatrix(params=dict(p, k_x=k)))
            for k in k_xs
        ]
    )


fig, axs = plt.subplots(2, 3, figsize=(2 * fig_width, 2 * fig_height))
plt.subplots_adjust(hspace=0.2)


def draw_circle(ax, r, x0, y0, angle, theta2, c="black", with_arrow_head=True, lw=1):
    # https://stackoverflow.com/a/38208040/3447047
    from numpy import radians as rad
    from matplotlib.patches import Arc, RegularPolygon

    ax.add_patch(
        Arc(
            [x0, y0],
            r,
            r,
            angle=angle,
            theta1=0,
            theta2=theta2,
            capstyle="round",
            linestyle="-",
            color=c,
            lw=lw,
        )
    )
    end_x = x0 + (r / 2) * np.cos(rad(theta2 + angle))
    end_y = y0 + (r / 2) * np.sin(rad(theta2 + angle))
    if with_arrow_head:
        ax.add_patch(
            RegularPolygon((end_x, end_y), 3, r / 9, rad(angle + theta2), color=c)
        )
    ax.set_xlim([x0 - r, y0 + r])
    ax.set_ylim([y0 - r, y0 + r])


axs[0][0].set_title(label="Insulator")
axs[0][1].set_title(label="QH")
axs[0][2].set_title(label="TI")


for i, ax in enumerate(axs.flatten()):
    if i in (3, 4, 5):
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_ylabel("$E$")
        ax.set_xlabel("$k_x$", labelpad=8)
        ax.set_ylim((-1.5, 1.5))
        ax.set_xlim((-1, 1))
    else:
        ax.set_xticks([])
        ax.set_yticks([])
        ax.axis("equal")
        ax.patch.set_facecolor((0.95, 0.95, 0.95))

r = 0.7
theta2 = 360 - 5

# Insulator
ax = axs[0][0]
for i in np.linspace(-1, 1, 3):
    for j in np.linspace(-1, 1, 3):
        draw_circle(ax, r, i, j, 0, theta2)
        ax.scatter(i, j, s=10, c="grey")
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)

# QH
ax = axs[0][1]
for i in np.linspace(-1, 1, 3):
    for j in np.linspace(-1, 1, 3):
        draw_circle(ax, r, i, j, 0, theta2)

for x in np.arange(-3, 3, r):
    draw_circle(ax, r, x, -2.7, 90, theta2)
    draw_circle(ax, r, x, 2.7, -90, theta2)

# B_perp indication
x, y = (-2.25, 1.7)
draw_circle(ax, 0.4, x, y, 360, 360, with_arrow_head=False, lw=0.5)
ax.scatter(x, y, c="k", s=27, marker="x", linewidth='0.5')
ax.text(
    x,
    y - 0.4,
    "$B_\perp$",
    verticalalignment="top",
    horizontalalignment="center",
    fontsize=10,
)

ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)


# TI
ax = axs[0][2]
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)
arrow_kwargs = dict(dy=0, head_width=0.2, head_length=0.2)

for s in [-1, 1]:
    ax.arrow(x=2 * s, y=1.5 * s, dx=-4 * s, **arrow_kwargs, color="C0")
    ax.arrow(x=-2 * s, y=1.9 * s, dx=4 * s, **arrow_kwargs, color="C1")

text_kwargs = dict(horizontalalignment="center", x=0)
ax.text(
    y=1.3,
    s=r"$\left| e,\uparrow \right>$",
    verticalalignment="top",
    color="C1",
    **text_kwargs
)
ax.text(
    y=-1.3,
    s=r"$\left| e,\downarrow \right>$",
    verticalalignment="bottom",
    color="C0",
    **text_kwargs
)

# Insulator
ax = axs[1][0]
ax.plot(k_xs, bands(p_triv), c="k", lw=1)

# QH
ax = axs[1][1]
E = np.sort(bands(p_topo), axis=1)[:, ::2]  # remove degenerate bands
E[:25, 19], E[:25, 20] = (
    copy(E[:25, 20]),
    copy(E[:25, 19]),
)  # make spin channels continuous

E_qh = copy(E)
E_qh[:, 19] = np.nan  # drop one spin channel for QH
ax.plot(k_xs, E_qh, c="k", lw=1)

# TI
ax = axs[1][2]
ax.plot(k_xs, E, c="k", lw=1)

plt.tight_layout()
plt.savefig("figures/compare-insulators.pdf", bbox_inches="tight", transparent=True)

# QHE plot

In [ ]:
import adaptive
adaptive.notebook_extension()

def qhe_hall_bar(L=50, W=10, w_lead=10, w_vert_lead=28):
    """Create a hall bar system. 

    Square lattice, one orbital per site.
    Returns kwant system.

    Arguments required in onsite/hoppings: 
        t, mu, mu_lead
    """

    L = 2 * (L // 2)
    W = 2 * (W // 2)
    w_lead = 2 * (w_lead // 2)
    w_vert_lead = 2 * (w_vert_lead // 2)

    # bar shape
    def bar(pos):
        (x, y) = pos
        return (x >= -L / 2 and x <= L / 2) and (y >= -W / 2 and y <= W / 2)

    # Onsite and hoppings
    def onsite(site, t, mu):
        (x, y) = site.pos
        return 4 * t - mu

    def hopping_Ax(site1, site2, t, B):
        x1, y1 = site1.pos
        x2, y2 = site2.pos
        return -t * np.exp(-0.5j * B * (x1 + x2) * (y1 - y2))

    def make_lead_hop_y(x0):
        def hopping_Ay(site1, site2, t, B):
            x1, y1 = site1.pos
            x2, y2 = site2.pos
            return -t * np.exp(-1j * B * x0 * (y1 - y2))

        return hopping_Ay

    def lead_hop_vert(site1, site2, t):
        return -t

    # Building system
    lat = kwant.lattice.square()
    syst = kwant.Builder()

    syst[lat.shape(bar, (0, 0))] = onsite
    syst[lat.neighbors()] = hopping_Ax

    # Attaching leads
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead = kwant.Builder(sym_lead)

    def lead_shape(pos):
        (x, y) = pos
        return -w_lead / 2 <= y <= w_lead / 2

    lead_onsite = lambda site, t, mu_lead: 4 * t - mu_lead

    sym_lead_vertical = kwant.TranslationalSymmetry((0, 1))
    lead_vertical1 = kwant.Builder(sym_lead_vertical)
    lead_vertical2 = kwant.Builder(sym_lead_vertical)

    def lead_shape_vertical1(pos):
        (x, y) = pos
        return -L / 4 - w_vert_lead / 2 <= x <= -L / 4 + w_vert_lead / 2

    def lead_shape_vertical2(pos):
        (x, y) = pos
        return +L / 4 - w_vert_lead / 2 <= x <= +L / 4 + w_vert_lead / 2

    lead_vertical1[lat.shape(lead_shape_vertical1, (-L / 4, 0))] = lead_onsite
    lead_vertical1[lat.neighbors()] = lead_hop_vert
    lead_vertical2[lat.shape(lead_shape_vertical2, (L / 4, 0))] = lead_onsite
    lead_vertical2[lat.neighbors()] = lead_hop_vert

    syst.attach_lead(lead_vertical1)
    syst.attach_lead(lead_vertical2)

    syst.attach_lead(lead_vertical1.reversed())
    syst.attach_lead(lead_vertical2.reversed())

    lead[lat.shape(lead_shape, (-1, 0))] = lead_onsite
    lead[lat.neighbors()] = make_lead_hop_y(-L / 2)

    syst.attach_lead(lead)

    lead = kwant.Builder(sym_lead)
    lead[lat.shape(lead_shape, (-1, 0))] = lead_onsite
    lead[lat.neighbors()] = make_lead_hop_y(L / 2)

    syst.attach_lead(lead.reversed())

    return syst


def calculate_sigmas(G):
    # reduce by one dimension G -> G[temp, temp]
    temp = [0, 1, 3, 4, 5]
    G = G[temp, :]
    G = G[:, temp]
    # invert R = G^-1
    # find out whether it is a numpy object
    r = np.linalg.inv(G)
    # Voltages follow: V = R I[temp]
    V = r @ np.array([0, 0, 0, 1, -1])
    # Completely solved the six terminal system.
    # Consider the 2x2 conductance now: Use I = sigma U
    E_x = V[1] - V[0]
    E_y = V[1] - V[3]
    # formula above
    sigma_xx = E_x / (E_x ** 2 + E_y ** 2)
    sigma_xy = E_y / (E_x ** 2 + E_y ** 2)
    return sigma_xx, sigma_xy


syst = qhe_hall_bar(L=60, W=100, w_lead=90, w_vert_lead=28).finalized()
p = dict(t=1.0, mu=0.3, mu_lead=0.3, B=None)
Bs = np.linspace(0.02, 0.15, 200)
num_leads = len(syst.leads)


def G(syst, p):
    smatrix = kwant.smatrix(syst, params=p)
    G = [
        [smatrix.transmission(i, j) for i in range(num_leads)] for j in range(num_leads)
    ]
    G -= np.diag(np.sum(G, axis=0))
    return calculate_sigmas(G)

def sigma_xy(B):
    return G(syst, dict(p, B=B))[1]

In [ ]:
learner = adaptive.Learner1D(sigma_xy, bounds=(0.02, 0.15))

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.npoints > 200)

In [ ]:
runner.live_info()

In [ ]:
Bs, sigmasxy = zip(*learner.data.items())

fig, ax = plt.subplots(1, 1, figsize=(1.5 * fig_width, 1.5 * fig_height))
Bs_inv = 1 / np.array(Bs)
i = np.argsort(Bs_inv)
ax.plot(Bs_inv[i], np.array(sigmasxy)[i], label=r"$\sigma_{xy}$", c="k")
ax.set_xlabel(r"$B_\perp^{-1}$ [a.u.]")
ax.set_ylabel(r"$G_\textrm{H}$ [$e^2/h$]")
ax.grid(which="major", axis="y")
plt.savefig("figures/qhe.pdf", bbox_inches="tight", transparent=True)

In [ ]:
Bs, sigmasxy = zip(*learner.data.items())

fig, ax = plt.subplots(1, 1, figsize=(2 * fig_width, 2 * fig_height))
Bs = np.array(Bs)
i = np.argsort(Bs)
ax.plot(Bs[i], 1 / np.array(sigmasxy)[i], label=r"$\sigma_{xy}$", c="k")
ax.set_xlabel(r"$B_\perp$ [a.u.]")
ax.set_ylabel(r"$R_\textrm{H}$ [$h/e^2$]")